# Experimental analyses for the week of January 8th

This week, I performed three sets of experiments, mainly focusing on different sizes of the "hard convolution" problem and how solution times to that problem were impacted by changes in the Swizzleflow code.

The "hard convolution" problem has the following loop body pseudocode
```
to_send = regs[?select(tid, i)]
received = _shuffle(to_send, ?swizzle(tid, i)]
out[tid] += received * weights[?swizzle(tid, i)]
```
The difference between this problem and the one presented in the Swizzle Inventor paper's benchmarks is that this problem introduces the extra complexity of needing to pick the swizzle for the weights.

This benchmark is problematic for Swizzleflow, as we saw in previous experiments, due to the lack of pruning on the weight swizzle. The initial search for the weight swizzle is performed independenly. From the perspective of that independent search, the search problem is of the form
$$\begin{bmatrix} 1 & 2 & 3\\
1 & 2 & 3\\
\vdots & \vdots &\vdots\\
1 & 2 & 3
\end{bmatrix} \to
\begin{bmatrix}
1 + 2 + 3\\
1 + 2 + 3\\
\vdots\\
1 + 2 + 3
\end{bmatrix}$$
where the intermediate steps only permute within the rows of the matrix.

This problem's solutions are any swizzle that forms a permutation, which, empirically, is all of them. This has caused significant explosions in synthesis time for the hard convolution benchmark, which I have examined and attempted to reduse as part of this week's experiments.

In [1]:
# initial setup
import sys
sys.path.append("../analysis")

In [2]:
import parsing
import extraction

from parsing import parse_file
from extraction import humanize_names, expand_target_checks, pull_spec_in

In [3]:
import pandas as pd
import numpy as np

In [4]:
def fetch(dataset):
    return humanize_names(parse_file(f"../results/{dataset}"))

In [5]:
# Files
hard_conv_times_raw = fetch("2020-01-06-hard-convolution-no-groups-search-only")
hard_conv_stats_raw = fetch("2020-01-06-hard-convolution-no-groups-more-stats")

symmetry_break_times_raw = fetch("2020-01-06-hard-convolution-no-groups-break-symmetry")
symmetry_break_stats_raw = fetch("2020-01-06-hard-convolution-no-groups-break-symmetry-stats")

no_lane1_prune_times_raw = fetch("2020-01-06-hard-convolution-no-groups-break-symmetry-with-naive-l1")

self_pair_last_times_raw = fetch("2020-01-07-symmetry-break-self-last")
self_pair_first_times_raw = fetch("2020-01-07-symmetry-break-self-first")
self_pair_last_stats_raw = fetch("2020-01-07-symmetry-break-self-last-stats")
self_pair_first_stats_raw = fetch("2020-01-08-symmetry-break-self-first-stats")

## Target checks statistics
Early this week, as I mentioned to you at the TA meeting, I introduced code that would track how many different pairs of terms were checked before a candidate node was rejected during pruning. Collecting this information significantly slowed the search, but did provide some insights into our code's performance.

To save on output size and storage space, and to make the data easier to intpretet, I decided to pre-bin the data by its first digit (preserving the 100s place on). That is, while rejections after 1 and 2 are counted seperately, the data point that a candidtae needed 11 or 12 rejections is counted under "10", since distinguishing between 11 and 12 is much less useful that between 1 and 2.

One of these insights was that, in many applications, like Trove, the first test, which checked $(x_0, x_0)$ or similar, _appeared_ to never reject a candidate. This led to one of my experiments, which involved moving that test to the end of each "row" of $(x_i, b)$. (This turned out not to have too much of an effect, as shown later).

Here are some almost representative examples of these pruning statistics. These are taken from a more recent experiment, which took place after the symmetry-breaking changes, due to the fact that the old results aren't output in a way that's compatible with my parsing code. However, they'll still serve to illustrate the main point.

In [6]:
self_pair_first_stats = extraction.search_stats(self_pair_first_stats_raw)
expand_target_checks(self_pair_first_stats['l1/trove-crc-3'], to_copy=["pruned", "name"])

,name,pruned,1,2,3,4,7,8,10,20,30,40,50,90
0,load_rep,0,0,0,0,0,0,0,0,0,0,0,0,0
1,row_xforms_no_group,28,0,10,2,0,0,2,10,0,2,0,1,1
2,row_rots_no_group,4,0,2,0,0,0,0,0,0,0,0,0,2
3,col_xforms_no_group,1982,0,1922,0,60,0,0,0,0,0,0,0,0
4,col_rots_no_group,62,62,0,0,0,0,0,0,0,0,0,0,0
5,row_xforms_no_group,58,0,30,0,20,2,0,2,2,0,2,0,0
6,row_rots_no_group,0,0,0,0,0,0,0,0,0,0,0,0,0
7,(last),0,0,0,0,0,0,0,0,0,0,0,0,0


and for a stencil

In [7]:
expand_target_checks(self_pair_first_stats['l1/1d-stencil'], to_copy=['pruned', 'name'])

,name,pruned,1,2,3,4,7,90,4700
0,load_trunc,0,0,0,0,0,0,0,0
1,reg_select_no_consts,13,3,3,0,1,2,2,2
2,col_xforms_no_group,991,0,961,30,0,0,0,0
3,col_rots_no_group,31,31,0,0,0,0,0,0
4,cond_keep_no_consts,0,0,0,0,0,0,0,0
5,(last),0,0,0,0,0,0,0,0


From these tables, we can see that, in general, $(x_0, x_0)$ is a poor candidate for pruning in many of our benchmarks, while the pairs $(x_0, x_1)$ and $(x_0, x_2)$ often do most of the work.

The main outlier appears _after_ the rotation steps, where the high prevalence of rejections on the first pair shows that the main conclusion used to prune in those phases is "there's no way to get $x_0$ where it needs to be". This is possible because, after those rotations, there are no ways to permute $x_0$ along the relevant axis.

I'm not showing the data for hard convolution here since the relevant part of that search is exactly the same as in the stencil example above.

### **Implications for sparse matrix design**
This data shows that, in a lot af cases, we'll only be accessing a few columns of the pruning matrix. Therefore, we'll want a matrix structure that has high cache locality when repeatedly accessing the same column.

One obvious approach here is an array of `BTreeSet`s, one per column. This should have reasonable lookup performance and good cache efficiency.

## Symmetry breaking experiments

The initial version fo the code, which is where I was last week is one where, in the pruning step, we test each pair of terms $(x_i, x_j)$, which means we redundantly test both $(a, b)$ and $(b, a)$, when $a != b$.

I didn't expect to see much difference in pruning times on benchmarks that had a lot of quick rejections of solutions (like the Swizzle Inventor benchmarks shown in the tables above), and so I started working on the hard convolution benchmark.

The experiments I performed were:
 - Breaking symmetry, where I'd only test the upper triangle of pairs of terms.That is, if $x_i$ was at the $k$th position of my list of terms, I'd only test pairs of it and elements in positinos $l \geq k$. I expected this to produce some noticable reduction in synthesis times on these hard benchmarks.
 - Moving the self test last. In the initial symmetry-broken code, the first pair we'd test for each $a$ is $(a, a)$. Since I knew this often wasn't the value I wanted to look at first. I checked what would happen if I moved that test to the end.
 
Additionally, while performing these tests, I decided to check what would happen if I removed the pruning tests from the brench of the convolution search where pruning always let every solution through. This would measure the performance impact of the wasted pruning.

In [8]:
hard_conv_names = list(hard_conv_times_raw.keys())
no_prune_names = list(no_lane1_prune_times_raw.keys())

In [9]:
time_table = pd.DataFrame({
    "Initial": extraction.search_info(hard_conv_times_raw)["time"],
    "Symmetry broken": extraction.search_info(symmetry_break_times_raw)["time"],
    "Self last": extraction.search_info(self_pair_last_times_raw)["time"][hard_conv_names]
})
time_table

,Initial,Symmetry broken,Self last
spec,,,
16x3,0.046519,0.034069,0.034072
32x3,0.705464,0.562484,0.596337
32x4,3.093710,2.361378,2.509919
32x5,9.772834,7.573347,7.478793


In [10]:
no_prune_time_table = pd.DataFrame({
    "Symmetry broken": extraction.search_info(no_lane1_prune_times_raw)["time"],
    "Self last": extraction.search_info(self_pair_last_times_raw)["time"][no_prune_names]
})
no_prune_time_table

,Symmetry broken,Self last
spec,,
16x3-no-prune,0.002833,0.002647
32x3-no-prune,0.014337,0.014490
32x4-no-prune,0.036207,0.038436
32x5-no-prune,0.067055,0.068569


The tables above lead us to the following conclusions:
1. Removing the redundant symmetrical checks has an effect on the time it takes to complete pruning on correct candidates when there are many such candidates.
2. There is a much more significant effect from removing the pruning on the unprunable section of the "hard convolution" benchmalk (I will explain why below)
3. Moving the self-test last does not have a significant effect on pruning time. This is likely because most pruning happens so quickly in the benchmarks we have (which also don't feature many copies of $x_0$, and often have just one) that small number of array lookups involved in that test is negligible.

As one additoal datapoint on the self-test-last movement, we can look at the data for the Swizzle Inventor benchmarks.

In [11]:
swinv_times_bench = pd.DataFrame({
    "Self first": extraction.search_info(self_pair_first_times_raw)["time"],
    "Self last": extraction.search_info(self_pair_last_times_raw)["time"]})
swinv_times_bench.dropna(inplace=True)
swinv_times_bench["Improvement"] = swinv_times_bench["Self last"] - swinv_times_bench["Self first"]
swinv_times_bench["% change"] = swinv_times_bench["Improvement"] / swinv_times_bench["Self first"]
swinv_times_bench

,Self first,Self last,Improvement,% change
l1/1d-conv,0.006049,0.006242,1.933940e-04,0.031971
l1/1d-stencil,0.002996,0.003394,3.985990e-04,0.133057
l1/2d-stencil-3,0.002158,0.002451,2.932790e-04,0.135933
l1/mult-32-with-4,0.002534,0.002229,-3.056940e-04,-0.120623
l1/trove-cr_sum-1,0.000174,0.000173,-9.280000e-07,-0.005346
l1/trove-cr_sum-2,0.006512,0.007765,1.253145e-03,0.192448
l1/trove-cr_sum-3,0.009758,0.010189,4.305850e-04,0.044125
l1/trove-cr_sum-4,0.123345,0.124042,6.975140e-04,0.005655
l1/trove-cr_sum-5,0.017044,0.015755,-1.289190e-03,-0.075637
l1/trove-cr_sum-7,0.023242,0.023404,1.621010e-04,0.006974


This table shows that, on many benchmarks, this change is a minor optimization at best, but that the position of the self pair is, fundamentally, not all that significant.

## Why did removing pruning help so much on hard convolution?

In the hard convolution benchmalk, there are 32 copies of each value in the target at each intermediate step.
That means, when we generate the explosion of, say $(x_0, x_1)$, there are $32^2 = 1024$ target (or source) location pairs for that pair.
Since the pruning test must evaluate all of them, for each of the $\binom{w}{2}$ pairs of values, we need to perform between $1024$ and $1024^2$ (since, at worst, the last source location pair is the one that can be moved) array lookups.

This number is per candidate solution.

Of which, between the two search steps in that process, there are hundreds.

This takes quite a long time to not prune, which is where our bad performance when leaving pruning for these search steps comes from.

One implication for this is what we may mant to (since we can, without sacrificing correctness) put an upper limit on how much pruning we do for each candidate.